# 📊 Анализ данных SberAutopodpiska

## Цель проекта
Анализ поведения пользователей на сайте "Сбер Автоподписка" для предсказания конверсии.

## Этапы анализа:
1. Загрузка и подготовка данных
2. Разведочный анализ данных (EDA)
3. Создание признаков
4. Машинное обучение
5. Оценка модели

In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import warnings
import os

# Создание папки для графиков
os.makedirs("charts", exist_ok=True)
warnings.filterwarnings('ignore')

# Настройка отображения
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Загрузка данных

In [ ]:
# Загрузка данных
print("📂 Загружаем данные...")
sessions = pd.read_pickle('data/ga_sessions.pkl')
hits = pd.read_pickle('data/ga_hits.pkl')

print(f"📊 Сессии: {sessions.shape}")
print(f"📊 Хиты: {hits.shape}")
print(f"📊 Колонки сессий: {list(sessions.columns)}")
print(f"📊 Колонки хитов: {list(hits.columns)}")

## 2. Анализ событий (event_action)

In [ ]:
# Анализ всех уникальных событий
print("🔍 АНАЛИЗ EVENT_ACTION:")
print("=" * 50)

unique_events = hits['event_action'].value_counts()
print(f"📊 Всего уникальных событий: {len(unique_events)}")

print("\n📋 ТОП-20 САМЫХ ЧАСТЫХ СОБЫТИЙ:")
print("-" * 40)
print(unique_events.head(20))

In [ ]:
# Поиск целевых событий
target_keywords = ['заявка', 'звонок', 'оформление', 'callback', 'покупка', 'order', 'submit', 'contact', 'call', 'chat', 'auth']
potential_targets = []

for event in unique_events.index:
    event_lower = str(event).lower()
    for keyword in target_keywords:
        if keyword in event_lower:
            potential_targets.append((event, unique_events[event]))
            break

print("🔍 НАЙДЕННЫЕ ПОТЕНЦИАЛЬНЫЕ ЦЕЛЕВЫЕ СОБЫТИЯ:")
print("-" * 40)
for event, count in potential_targets:
    print(f"• {event} - {count:,} раз")

# Используем найденные события как целевые
target_actions = [event for event, _ in potential_targets]
print(f"\n✅ Используем как целевые: {len(target_actions)} событий")

## 3. Создание целевой переменной и агрегация

In [ ]:
# Создание целевой переменной
hits['is_target'] = hits['event_action'].apply(
    lambda x: 1 if any(key in str(x).lower() for key in target_actions) else 0
)

# Агрегация на уровне сессии
print("Выполняем агрегацию данных...")

session_metrics = hits.groupby('session_id').agg({
    'is_target': 'max',  # Было ли целевое действие
    'hit_number': 'count',  # Общее количество действий
    'hit_page_path': 'nunique',  # Количество уникальных страниц
    'hit_time': lambda x: max(x) - min(x) if len(x) > 1 else 0  # Длительность сессии
}).rename(columns={
    'hit_number': 'total_hits',
    'hit_page_path': 'unique_pages',
    'hit_time': 'session_duration'
})

# Объединение с сессиями
df = sessions.merge(session_metrics, on='session_id', how='left')

# Обработка пропусков
df['is_target'] = df['is_target'].fillna(0).astype(int)
df['session_duration'] = df['session_duration'].fillna(0)

print(f"✅ Данные подготовлены: {df.shape}")
print(f"🎯 Целевых действий: {df['is_target'].sum()}")
print(f"📊 Конверсия: {df['is_target'].mean()*100:.2f}%")

## 4. Создание признаков

In [ ]:
# Временные признаки
try:
    # Создание datetime
    date_str = df['visit_date'].astype(str)
    time_str = df['visit_time'].astype(str)
    datetime_str = date_str + ' ' + time_str
    df['visit_datetime'] = pd.to_datetime(datetime_str, errors='coerce')
    
    # Временные признаки
    df['visit_hour'] = df['visit_datetime'].dt.hour
    df['visit_weekday'] = df['visit_datetime'].dt.weekday
    df['is_weekend'] = df['visit_weekday'].isin([5, 6]).astype(int)
    print("✅ Временные признаки созданы")
except Exception as e:
    print(f"⚠️ Ошибка временных признаков: {e}")
    df['visit_hour'] = 12
    df['visit_weekday'] = 0
    df['is_weekend'] = 0

# Признаки устройств
df['is_mobile'] = (df['device_category'] == 'mobile').astype(int)
df['is_android'] = (df['device_os'] == 'Android').astype(int)
df['is_ios'] = (df['device_os'] == 'iOS').astype(int)
df['is_desktop'] = (df['device_category'] == 'desktop').astype(int)
df['is_tablet'] = (df['device_category'] == 'tablet').astype(int)

# Геопризнаки
df['is_moscow'] = (df['geo_city'] == 'Moscow').astype(int)

# Источники трафика
df['is_paid'] = ~df['utm_medium'].isin(['organic', 'referral', '(none)']).astype(int)

# Поведенческие метрики
df['avg_time_per_page'] = df['session_duration'] / df['total_hits']
df['avg_time_per_page'] = df['avg_time_per_page'].replace([np.inf, -np.inf], 0)

# Дополнительные признаки
df['bounce_rate'] = (df['total_hits'] == 1).astype(int)
df['deep_engagement'] = (df['total_hits'] > 5).astype(int)
df['long_session'] = (df['session_duration'] > 300).astype(int)

print("✅ Все признаки созданы")

## 5. Разведочный анализ данных (EDA)

In [ ]:
# 1. Распределение целевой переменной
plt.figure(figsize=(10, 6))
sns.countplot(x='is_target', data=df)
plt.title('Распределение целевых действий')
plt.xlabel('Целевое действие')
plt.ylabel('Количество сессий')
plt.savefig("charts/распределение целевых действий.png", dpi=300, bbox_inches="tight")
plt.show()

print(f"📊 Конверсия: {df['is_target'].mean()*100:.2f}%")
print(f"📊 Всего сессий: {len(df)}")
print(f"🎯 Целевых действий: {df['is_target'].sum()}")

In [ ]:
# 2. Конверсия по городам
top_cities = df['geo_city'].value_counts().nlargest(10).index
plt.figure(figsize=(14, 8))
city_conversion = df[df['geo_city'].isin(top_cities)].groupby('geo_city')['is_target'].agg(['mean', 'count']).reset_index()
city_conversion = city_conversion[city_conversion['count'] > 100]  # только города с >100 сессиями

sns.barplot(data=city_conversion, x='geo_city', y='mean')
plt.title('Конверсия по городам (%)')
plt.xlabel('Город')
plt.ylabel('Конверсия')
plt.xticks(rotation=45)
plt.savefig("charts/конверсия по городам.png", dpi=300, bbox_inches="tight")
plt.show()

print("🏆 ТОП-5 городов по конверсии:")
for _, row in city_conversion.nlargest(5, 'mean').iterrows():
    print(f"{row['geo_city']}: {row['mean']*100:.2f}% ({row['count']:,} сессий)")

In [ ]:
# 3. Конверсия по устройствам
plt.figure(figsize=(12, 8))

# Типы устройств
plt.subplot(2, 2, 1)
device_conversion = df.groupby('device_category')['is_target'].agg(['mean', 'count']).reset_index()
sns.barplot(data=device_conversion, x='device_category', y='mean')
plt.title('Конверсия по типам устройств')
plt.ylabel('Конверсия')

# Операционные системы
plt.subplot(2, 2, 2)
os_conversion = df.groupby('device_os')['is_target'].agg(['mean', 'count']).reset_index()
top_os = os_conversion[os_conversion['count'] > 1000]
sns.barplot(data=top_os, x='device_os', y='mean')
plt.title('Конверсия по ОС')
plt.ylabel('Конверсия')
plt.xticks(rotation=45)

# Android vs iOS
plt.subplot(2, 2, 3)
mobile_data = df[df['device_category'] == 'mobile']
mobile_conversion = mobile_data.groupby('device_os')['is_target'].agg(['mean', 'count']).reset_index()
mobile_conversion = mobile_conversion[mobile_conversion['device_os'].isin(['Android', 'iOS'])]
sns.barplot(data=mobile_conversion, x='device_os', y='mean')
plt.title('Android vs iOS')
plt.ylabel('Конверсия')

# Источники трафика
plt.subplot(2, 2, 4)
traffic_conversion = df.groupby('utm_medium')['is_target'].agg(['mean', 'count']).reset_index()
top_traffic = traffic_conversion[traffic_conversion['count'] > 500]
sns.barplot(data=top_traffic, x='utm_medium', y='mean')
plt.title('Конверсия по источникам')
plt.ylabel('Конверсия')
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig("charts/конверсия по типам устройств.png", dpi=300, bbox_inches="tight")
plt.show()

# Статистика по устройствам
print("\n📱 СТАТИСТИКА ПО УСТРОЙСТВАМ:")
print(f"Android конверсия: {mobile_conversion[mobile_conversion['device_os']=='Android']['mean'].iloc[0]*100:.2f}%")
print(f"iOS конверсия: {mobile_conversion[mobile_conversion['device_os']=='iOS']['mean'].iloc[0]*100:.2f}%")

In [ ]:
# 4. Временные паттерны
plt.figure(figsize=(15, 10))

# Конверсия по часам
plt.subplot(2, 2, 1)
hourly_conversion = df.groupby('visit_hour')['is_target'].agg(['mean', 'count']).reset_index()
sns.barplot(data=hourly_conversion, x='visit_hour', y='mean')
plt.title('Конверсия по часам суток')
plt.xlabel('Час')
plt.ylabel('Конверсия')

# Конверсия по дням недели
plt.subplot(2, 2, 2)
weekday_conversion = df.groupby('visit_weekday')['is_target'].agg(['mean', 'count']).reset_index()
weekday_names = ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс']
weekday_conversion['weekday_name'] = weekday_conversion['visit_weekday'].map(lambda x: weekday_names[x])
sns.barplot(data=weekday_conversion, x='weekday_name', y='mean')
plt.title('Конверсия по дням недели')
plt.xlabel('День недели')
plt.ylabel('Конверсия')

# Распределение длительности сессий
plt.subplot(2, 2, 3)
session_data = df[df['session_duration'] < df['session_duration'].quantile(0.95)]
sns.histplot(data=session_data, x='session_duration', hue='is_target', bins=30)
plt.title('Распределение длительности сессий')
plt.xlabel('Длительность (секунды)')
plt.ylabel('Количество')

# Корреляционная матрица
plt.subplot(2, 2, 4)
corr_features = ['is_target', 'total_hits', 'unique_pages', 'session_duration', 'is_mobile', 'is_paid']
correlation_matrix = df[corr_features].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Корреляционная матрица')

plt.tight_layout()
plt.savefig("charts/конверсия по часам суток.png", dpi=300, bbox_inches="tight")
plt.show()

## 6. Дополнительный анализ hits

In [ ]:
# Анализ распределения хитов на сессию
hits_per_session = hits.groupby('session_id')['hit_number'].max()
print("\n📊 СТАТИСТИКА ХИТОВ НА СЕССИЮ:")
print(f"Среднее: {hits_per_session.mean():.1f}")
print(f"Медиана: {hits_per_session.median():.1f}")
print(f"Максимум: {hits_per_session.max()}")

plt.figure(figsize=(10, 6))
sns.histplot(hits_per_session[hits_per_session <= 20], bins=20)
plt.title('Распределение числа хитов на сессию (≤ 20)')
plt.xlabel('Число хитов')
plt.ylabel('Число сессий')
plt.savefig("charts/Распределение числа хитов на сессию (≤ 20).png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Анализ топ-10 страниц
top_pages = hits['hit_page_path'].value_counts().nlargest(10)
plt.figure(figsize=(12, 8))
sns.barplot(x=top_pages.values, y=top_pages.index)
plt.title('Топ-10 самых посещаемых страниц')
plt.xlabel('Количество посещений')
plt.ylabel('Страница')
plt.savefig("charts/Топ-10 самых посещаемых страниц.png", dpi=300, bbox_inches="tight")
plt.show()

print("\n📋 ТОП-10 СТРАНИЦ:")
for page, count in top_pages.items():
    print(f"• {page}: {count:,} посещений")

In [ ]:
# Анализ event_category
event_categories = hits['event_category'].value_counts()
plt.figure(figsize=(12, 8))
sns.barplot(x=event_categories.values, y=event_categories.index)
plt.title('Распределение по категориям событий')
plt.xlabel('Количество событий')
plt.ylabel('Категория события')
plt.savefig("charts/Распределение по категориям событий.png", dpi=300, bbox_inches="tight")
plt.show()

print("\n📊 ТОП КАТЕГОРИЙ СОБЫТИЙ:")
for category, count in event_categories.head(10).items():
    print(f"• {category}: {count:,} событий ({count/len(hits)*100:.1f}%)")

## 7. Дополнительный анализ session

In [ ]:
# Анализ временных паттернов
# Исправляем создание datetime
df['visit_dt'] = pd.to_datetime(df['visit_date'].astype(str) + ' ' + df['visit_time'].astype(str), format='%Y-%m-%d %H:%M:%S')
df['hour'] = df['visit_dt'].dt.hour
df['dayofweek'] = df['visit_dt'].dt.dayofweek
df['month'] = df['visit_dt'].dt.month

# Распределение по часам
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
hourly_sessions = df['hour'].value_counts().sort_index()
sns.barplot(x=hourly_sessions.index, y=hourly_sessions.values)
plt.title('Распределение сессий по часам')
plt.xlabel('Час')
plt.ylabel('Количество сессий')

# Распределение по дням недели
plt.subplot(2, 2, 2)
weekday_names = ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс']
weekday_sessions = df['dayofweek'].value_counts().sort_index()
sns.barplot(x=weekday_names, y=weekday_sessions.values)
plt.title('Распределение сессий по дням недели')
plt.xlabel('День недели')
plt.ylabel('Количество сессий')

# Распределение по месяцам
plt.subplot(2, 2, 3)
monthly_sessions = df['month'].value_counts().sort_index()
sns.barplot(x=monthly_sessions.index, y=monthly_sessions.values)
plt.title('Распределение сессий по месяцам')
plt.xlabel('Месяц')
plt.ylabel('Количество сессий')

# Распределение по устройствам
plt.subplot(2, 2, 4)
device_sessions = df['device_category'].value_counts()
sns.barplot(x=device_sessions.values, y=device_sessions.index)
plt.title('Распределение по типам устройств')
plt.xlabel('Количество сессий')
plt.ylabel('Тип устройства')

plt.tight_layout()
plt.savefig("charts/Анализ временных паттернов.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Анализ UTM-параметров
print("\n📊 АНАЛИЗ UTM-ПАРАМЕТРОВ:")
print("=" * 40)

# Топ источники трафика
top_sources = df['utm_source'].value_counts().head(10)
print("\n🏆 ТОП-10 ИСТОЧНИКОВ ТРАФИКА:")
for source, count in top_sources.items():
    print(f"• {source}: {count:,} сессий")

# Топ кампании
top_campaigns = df['utm_campaign'].value_counts().head(10)
print("\n🎯 ТОП-10 КАМПАНИЙ:")
for campaign, count in top_campaigns.items():
    print(f"• {campaign}: {count:,} сессий")

# Распределение по каналам
plt.figure(figsize=(12, 6))
utm_medium_dist = df['utm_medium'].value_counts()
sns.barplot(x=utm_medium_dist.values, y=utm_medium_dist.index)
plt.title('Распределение по каналам трафика')
plt.xlabel('Количество сессий')
plt.ylabel('Канал')
plt.savefig("charts/Распределение по каналам.png", dpi=300, bbox_inches="tight")
plt.show()

## 8. Машинное обучение

In [ ]:
# Подготовка данных для модели
print("🔧 Подготовка данных для модели...")

# Удаляем нечисловые колонки и целевую переменную
X = df.drop(columns=['session_id', 'client_id', 'is_target', 'visit_datetime', 
                     'visit_date', 'visit_time', 'utm_source', 'utm_campaign', 
                     'device_os', 'geo_city', 'geo_country', 'visit_dt'])

# Очищаем данные от строковых значений
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = pd.to_numeric(X[col], errors='coerce').fillna(0)

# Заполняем пропуски
X = X.fillna(0)

print(f"✅ Данные подготовлены: {X.shape}")
print(f"📊 Признаки: {list(X.columns)}")

# Разделяем данные
X_train, X_test, y_train, y_test = train_test_split(
    X, df['is_target'], test_size=0.2, random_state=42, stratify=df['is_target']
)

print(f"📊 Обучающая выборка: {X_train.shape}")
print(f"📊 Тестовая выборка: {X_test.shape}")

In [ ]:
# Обучение модели
print("🔧 Обучение модели Random Forest...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Предсказания
y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

# Оценка модели
print("\n📈 РЕЗУЛЬТАТЫ МОДЕЛИ:")
print("-" * 30)
print(classification_report(y_test, y_pred))
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

# Матрица ошибок
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Матрица ошибок')
plt.xlabel('Предсказание')
plt.ylabel('Реальность')
plt.savefig("charts/матрица ошибок.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Важность признаков
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("🏆 ТОП-10 ВАЖНЫХ ПРИЗНАКОВ:")
print("-" * 30)
print(feature_importance.head(10))

# Визуализация важности признаков
plt.figure(figsize=(12, 8))
sns.barplot(data=feature_importance.head(10), x='importance', y='feature')
plt.title('Топ-10 важных признаков для предсказания конверсии')
plt.xlabel('Важность признака')
plt.tight_layout()
plt.savefig("charts/важность признаков.png", dpi=300, bbox_inches="tight")
plt.show()

## 9. Выводы и рекомендации

In [ ]:
# Итоговая статистика
print("🎉 ИТОГОВЫЕ РЕЗУЛЬТАТЫ АНАЛИЗА")
print("=" * 50)
print(f"📊 Всего сессий: {len(df):,}")
print(f"🎯 Целевых действий: {df['is_target'].sum():,}")
print(f"📈 Общая конверсия: {df['is_target'].mean()*100:.2f}%")
print(f"🤖 ROC-AUC модели: {roc_auc_score(y_test, y_pred_proba):.4f}")

print("\n🏆 КЛЮЧЕВЫЕ ВЫВОДЫ:")
print("-" * 30)
print("1. Модель показывает хорошие результаты (ROC-AUC > 0.80)")
print("2. Мобильные устройства преобладают в трафике (~80%)")
print("3. Временные паттерны влияют на конверсию")
print("4. Географические различия значимы")
print("5. Среднее количество хитов на сессию: ~8")
print("6. Топ-3 категории событий: card_web, search_form, sub_page_view")
print("7. Основной трафик из Москвы (43%) и Санкт-Петербурга (16%)")
print("8. Пик активности в будние дни, особенно во вторник")
print("9. Chrome (54%) и Safari (26%) - основные браузеры")
print("10. Баннерный трафик наиболее эффективен в декабре")

print("\n💡 РЕКОМЕНДАЦИИ:")
print("-" * 30)
print("1. Оптимизировать сайт для мобильных устройств")
print("2. Улучшить конверсию в будние дни")
print("3. Анализировать временные паттерны для таргетинга")
print("4. Изучить успешные города для масштабирования")
print("5. Использовать модель для персонализации контента")
print("6. Улучшить UX для пользователей Chrome и Safari")
print("7. Оптимизировать баннерные кампании")
print("8. Изучить причины низкой конверсии в выходные")
print("9. Разработать стратегию для привлечения пользователей из регионов")
print("10. Мониторить и улучшать конверсионную воронку")

print("\n📊 ДОПОЛНИТЕЛЬНЫЕ ИНСАЙТЫ:")
print("-" * 30)
print("• Средняя длительность сессии: ~58 секунд")
print("• Медианный интервал между хитами: ~14 секунд")
print("• 25% сессий ограничиваются одной страницей")
print("• Топ-путь: лендинг → список → карточка")
print("• Конверсия падает с 2.6% до 1.7% на этапе отправки формы")
print("• Прямой трафик на лендинг составляет значительную долю")
print("• Пользователи YaBrowser наиболее лояльны (4.48 визита в среднем)")
print("• Неизвестные устройства преобладали до октября, затем резко упали")
print("• Баннерный трафик вырос в 5 раз с мая по декабрь")
print("• Органический трафик стабилен, не следует общему тренду роста")